In [16]:
import random
import torch
import numpy as np

In [17]:
torch.manual_seed(0)
torch.cuda.manual_seed(0)
np.random.seed(0)
random.seed(0)

In [18]:
from esol_utils import load_esol_data

(X_train, X_valid, X_test, y_train, y_valid, y_test, scaler) = load_esol_data()

Number of generated molecular descriptors: 217
Number of molecular descriptors without invalid values: 217


In [19]:
import os
import torch
import wandb
from torch import nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from pytorch_lightning.loggers import WandbLogger

In [29]:
class NeuralNetwork(pl.LightningModule):
    def __init__(self, input_dim, hdden_dim, train_data, val_data, test_data, batch_size=254, lr=1e-3):
        super().__init__()
        self.lr = lr
        self.train_data = train_data
        self.val_data = val_data
        self.test_data = test_data
        self.batch_size = batch_size

        self.model = nn.Sequential(
            nn.Linear(input_dim, hdden_dim),
            nn.ReLU(),
            nn.Linear(hdden_dim, hdden_dim),
            nn.ReLU(),
            nn.Linear(hdden_dim, 1)
        )

    def training_step(self, batch, batch_idx):
        x, y = batch
        z = self.model(x)
        loss = F.mse_loss(z, y)
        self.log("Train loss", loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        z = self.model(x)
        loss = F.mse_loss(z, y) 
        self.log("Valid MSE", loss)
        
    def test_step(self, batch, batch_idx):
        x, y = batch
        z = self.model(x)
        loss = F.mse_loss(z, y) 
        self.log("Test MSE", loss)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(
            self.parameters(),
            lr=self.lr
        )
        return optimizer
    
    def forward(self, x):
        return self.model(x).flatten()
    
    def train_dataloader(self):
        return DataLoader(self.train_data, batch_size=self.batch_size, shuffle=True)
    
    def val_dataloader(self):
        return DataLoader(self.val_data, batch_size=self.batch_size, shuffle=False)
    
    def test_dataloader(self):
        return DataLoader(self.test_data, batch_size=self.batch_size, shuffle=False)

In [21]:
from torch.utils.data import Dataset

class ESOLDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, idx):

        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        X_ = torch.as_tensor(self.X[idx].astype(np.float32))
        y_ = torch.as_tensor(self.y[idx].astype(np.float32).reshape(-1))
        
        return X_, y_

In [22]:
train_data = ESOLDataset(X_train, y_train)
val_data = ESOLDataset(X_valid, y_valid)
test_data = ESOLDataset(X_test, y_test)

In [23]:
wandb.init(project="nn-solubility",
           config={
               "batch_size": 32,
               "learning_rate": 0.001,
               "hidden_size": 512,
               "max_epochs": 100
           })

In [ ]:
nn_model = NeuralNetwork(
    input_dim = X_train.shape[1],
    hdden_dim = wandb.config["hidden_size"],
    train_data = train_data,
    val_data = val_data,
    test_data = test_data,
    lr = wandb.config["learning_rate"],
    batch_size=wandb.config["batch_size"]
)

wandb_logger = WandbLogger()
trainer = pl.Trainer(
    max_epochs = wandb.config["max_epochs"],
    logger = wandb_logger
)

trainer.fit(
    model=nn_model,
)

results = trainer.test(ckpt_path="best")
wandb.finish()

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name  | Type       | Params | Mode  | FLOPs
-----------------------------------------------------
0 | model | Sequential | 374 K  | train | 0    
-----------------------------------------------------
374 K     Trainable params
0         Non-trainable params
374 K     Total params
1.499     Total estimated model params size (MB)
6         Modules in train mode
0         Modules in eval mode
0         Total Flops


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\aryan\AppData\Local\Programs\Python\Python313\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:434: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
c:\Users\aryan\AppData\Local\Programs\Python\Python313\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:434: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
c:\Users\aryan\AppData\Local\Programs\Python\Python313\Lib\site-packages\pytorch_lightning\loops\fit_loop.py:317: The number of training batches (23) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.
Restoring states from the checkpoint path at .\lightning_logs\sbr5pasb\checkpoints\epoch=99-step=2300.ckpt
Loaded model weights from the checkpoint at .\lightning_logs\sbr5pasb\checkpoints\epoch=99-step=2300.ckpt
c:\Users\aryan\AppData\Local\Programs\Python\Python313\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:434: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        Test MSE            0.44986769556999207
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


Test MSE,▁
Train loss,█▅▃▃▄▃▃▃▂▂▃▂▂▂▂▂▁▂▁▁▁▁▂▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁
Valid MSE,█▃▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▂▂▂▃▃▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇██
trainer/global_step,▁▁▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇████
Test MSE,0.44987
Train loss,0.09469
Valid MSE,0.33912
epoch,100
trainer/global_step,2300


In [31]:
test_mse = results[0]["Test MSE"]
test_rmse = test_mse ** 0.5
print(f"\nANN model performance: RMSE on test set = {test_rmse:.4f}\n")


ANN model performance: RMSE on test set = 0.6707



In [39]:
#Increasing batch size, adding one more hidden layer, decreasing learning rate, increasing max epochs and adding early stopping due to higher number of epochs

from pytorch_lightning.callbacks import EarlyStopping

class NeuralNetwork(pl.LightningModule):
    def __init__(self, input_dim, hdden_dim, train_data, val_data, test_data, batch_size=254, lr=1e-3):
        super().__init__()
        self.lr = lr
        self.train_data = train_data
        self.val_data = val_data
        self.test_data = test_data
        self.batch_size = batch_size

        self.model = nn.Sequential(
            nn.Linear(input_dim, hdden_dim),
            nn.ReLU(),
            nn.Linear(hdden_dim, hdden_dim),
            nn.ReLU(),
            nn.Linear(hdden_dim, hdden_dim),
            nn.ReLU(),
            nn.Linear(hdden_dim, 1)
        )

    def training_step(self, batch, batch_idx):
        x, y = batch
        z = self.model(x)
        loss = F.mse_loss(z, y)
        self.log("Train loss", loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        z = self.model(x)
        loss = F.mse_loss(z, y) 
        self.log("Valid MSE", loss)
        
    def test_step(self, batch, batch_idx):
        x, y = batch
        z = self.model(x)
        loss = F.mse_loss(z, y) 
        self.log("Test MSE", loss)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(
            self.parameters(),
            lr=self.lr
        )
        return optimizer
    
    def forward(self, x):
        return self.model(x).flatten()
    
    def train_dataloader(self):
        return DataLoader(self.train_data, batch_size=self.batch_size, shuffle=True)
    
    def val_dataloader(self):
        return DataLoader(self.val_data, batch_size=self.batch_size, shuffle=False)
    
    def test_dataloader(self):
        return DataLoader(self.test_data, batch_size=self.batch_size, shuffle=False)

wandb.init(project="nn-solubility",
           config={
               "batch_size": 64,
               "learning_rate": 0.0001,
               "hidden_size": 512,
               "max_epochs": 500
           })



# --- Add Early Stopping Callback ---
early_stop = EarlyStopping(
    monitor="Valid MSE",
    patience=50,
    mode="min"
)

# --- Initialize Model ---
nn_model = NeuralNetwork(
    input_dim = X_train.shape[1],
    hdden_dim = wandb.config["hidden_size"],
    train_data = train_data,
    val_data = val_data,
    test_data = test_data,
    lr = wandb.config["learning_rate"],
    batch_size = wandb.config["batch_size"]
)

# --- WandB Logger ---
wandb_logger = WandbLogger()

# --- Trainer with EarlyStopping ---
trainer = pl.Trainer(
    max_epochs = wandb.config["max_epochs"],
    logger = wandb_logger,
    callbacks = [early_stop],              # <--- added
    accelerator = "auto",                  # <--- auto GPU/CPU
    devices = "auto"
)

# --- Training ---
trainer.fit(
    model = nn_model
)

# --- Testing using BEST checkpoint ---
results = trainer.test(ckpt_path="best")   # <--- uses best model

# --- Finish WandB run ---
wandb.finish()


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name  | Type       | Params | Mode  | FLOPs
-----------------------------------------------------
0 | model | Sequential | 637 K  | train | 0    
-----------------------------------------------------
637 K     Trainable params
0         Non-trainable params
637 K     Total params
2.550     Total estimated model params size (MB)
8         Modules in train mode
0         Modules in eval mode
0         Total Flops


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Restoring states from the checkpoint path at .\lightning_logs\i7q23qqs\checkpoints\epoch=161-step=1944.ckpt
Loaded model weights from the checkpoint at .\lightning_logs\i7q23qqs\checkpoints\epoch=161-step=1944.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        Test MSE            0.38525307178497314
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


Test MSE,▁
Train loss,█▄▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Valid MSE,█▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▅▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇█
trainer/global_step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
Test MSE,0.38525
Train loss,0.06266
Valid MSE,0.3678
epoch,162
trainer/global_step,1944


In [42]:
# --- Check why training ended ---
if trainer.should_stop:
    print("Training stopped early due to convergence (EarlyStopping triggered).")
else:
    print("Training ended because max_epochs was reached.")

test_mse = results[0]["Test MSE"]
test_rmse = test_mse ** 0.5
print(f"\nANN model performance: RMSE on test set = {test_rmse:.4f}\n")

Training stopped early due to convergence (EarlyStopping triggered).

ANN model performance: RMSE on test set = 0.6207

